In [1]:
import os
import cv2
import imutils
import tflearn
import pyautogui
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.utils import shuffle
from tflearn.layers.estimator import regression
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected

C:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Python310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def Add_Data(paths):
    images, labels = [], []
    number_of_datasets = len(paths)
    c = -1
    for i in paths:
        c += 1
        files = os.listdir(i)
        #num_images[i] = len()
        for j in files: 
            images.append(cv2.cvtColor(cv2.imread(i + "/" + j), cv2.COLOR_BGR2GRAY).reshape(89, 100, 1))
            z = [0 for i in range(number_of_datasets)]
            z[c] = 1
            labels.append(z)
    return images, labels

In [3]:
def CreateModel(number_of_datasets, shape_input = [None,89,100,1] ):
    #Number of layers are changed for testing
    tf.compat.v1.reset_default_graph()
    net = input_data(shape = shape_input, name='input')
    
    net = conv_2d(net, 32, 2, activation='relu')
    net = max_pool_2d(net, 2)
    
    net = conv_2d(net, 64, 2, activation='relu')
    net = max_pool_2d(net,2)

    net = conv_2d(net, 64, 2, activation='relu')
    net = max_pool_2d(net,2)

    net = conv_2d(net, 128, 2, activation='relu')
    net = max_pool_2d(net,2)

    net = conv_2d(net, 128, 2, activation='relu')
    net = max_pool_2d(net,2)

    net = conv_2d(net, 256, 2, activation='relu')
    net = max_pool_2d(net, 2)

    net = conv_2d(net, 256, 2, activation='relu')
    net = max_pool_2d(net, 2)

    net = conv_2d(net, 128, 2, activation='relu')
    net = max_pool_2d(net, 2)

    net = conv_2d(net, 64, 2, activation='relu')
    net = max_pool_2d(net, 2)

    net = fully_connected(net, 1000, activation='relu')
    net = dropout(net, 0.75)

    net = fully_connected(net, number_of_datasets, activation='softmax')

    net = regression(net, optimizer='adam', learning_rate= 1e-3, loss='categorical_crossentropy', name='regression')

    model = tflearn.DNN(net,tensorboard_verbose=0)

    return model

In [4]:
def ChooseAction1(selected_calss):
  if selected_calss == 0:
    #Move Down
    x = list(pyautogui.position())
    pyautogui.moveTo(x[0], x[1]+40, duration = 1)
  elif selected_calss == 1:
    #Move Up
    x = list(pyautogui.position())
    pyautogui.moveTo(x[0], x[1]-40, duration = 1)
  elif selected_calss == 2:
    #Move Left
    x = list(pyautogui.position())
    pyautogui.moveTo(x[0]-40, x[1], duration = 1)
  elif selected_calss == 3:
    #Move Right                       
    x = list(pyautogui.position())
    pyautogui.moveTo(x[0]+40, x[1], duration = 1)
  elif selected_calss == 4:
    #Click 
    x = list(pyautogui.position())
    pyautogui.click(x[0], x[1])

In [5]:
def ChooseAction2(selected_calss):
  if selected_calss == 0:
    pyautogui.press('volumeup')
  else:
    pyautogui.press('volumedown')

In [6]:
def Test_Model(model, actions = 2, class_names = ['Swing','Swag'], Number_of_Frames = 30):
  sr = False #Flag to start Recording
  checker = None
  frames, image_index= 0, 0
  start_index = 0
  camera = cv2.VideoCapture(0)
  pt,  pb = 10, 225
  pr, pl = 350, 590

  while(1):
      (g, f) = camera.read()
      if g:
          f = cv2.flip(imutils.resize(f, width=700), 1)
          (height, width) = f.shape[:2]
          img_cpy = f.copy()
          gray = cv2.GaussianBlur(cv2.cvtColor(f[pt:pb, pr:pl], cv2.COLOR_BGR2GRAY), (7, 7), 0)
          

          if frames < Number_of_Frames:
              if checker is None:
                   checker = gray.copy().astype("float")
              else:
                  cv2.accumulateWeighted(gray, checker, 0.5)
          else:
              y = cv2.threshold(cv2.absdiff(checker.astype("uint8"), gray), 25, 255, cv2.THRESH_BINARY)[1]
              (x, _) = cv2.findContours(y.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
              if 0 != len(x):
                h = (y, max(x, key=cv2.contourArea))
                if h is not None:
                  (t, s) = h
                  cv2.drawContours(img_cpy, [(pr, pt) + s], -1, (0, 0, 255))
                  if sr == True:
                    cv2.imwrite('x.png', t)
                    image = Image.open('x.png')
                    image = image.resize((100, int((float(image.size[1])*float((100/float(image.size[0])))))), Image.ANTIALIAS)
                    image.save('x.png')
                    image2 = cv2.imread('x.png')
                    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
                    selection = model.predict([image2.reshape(89, 100, 1)])
                    len_selection = len(selection[0])
                    total_selection = 0
                    for _ in range(len_selection):
                        total_selection += selection[0][_]
                    selected_calss = np.argmax(selection)
                    selected_calss_percentage = (np.amax(selection) / total_selection)
                    Class_Window = np.zeros((200,520,3), np.uint8)
                    cv2.putText(Class_Window,"Selected Class : " + class_names[selected_calss], (20, 20), cv2.FONT_ITALIC, 1,(255, 255, 255),2)
                    cv2.putText(Class_Window,"Accuracy : " + str(100 *selected_calss_percentage) + '%', (20, 80), cv2.FONT_ITALIC, 1,(255, 255, 255),2)
                    cv2.imshow("Class Info", Class_Window)
                    if selected_calss_percentage > 0.995:
                      if actions == 1:
                        k2 = cv2.waitKey(1) & 0xFF
                        if k2 == ord("d"):
                          ChooseAction1(selected_calss)
                      else:
                        ChooseAction2(selected_calss)
                  cv2.imshow("Gray Image", t)

          cv2.rectangle(img_cpy, (pl, pt), (pr, pb), (0, 255, 0), 2)
          frames = frames + 1

          cv2.imshow("Data Collection Window", img_cpy)
          k = cv2.waitKey(1) & 0xFF

          if  k == ord("q"):
              break
          elif k == ord("a"):
              sr = True
      else:
          break
  camera.release()
  cv2.destroyAllWindows()

**Train Gesturized Mouse Dataset**

In [10]:
# Add Data for Gesturized Mouse
Train_Paths = ["Dataset/SwingImages", "Dataset/PalmImages", "Dataset/peace", "Dataset/Swag", "Dataset/FistImages"]
Test_Paths = ["Dataset/SwingTest", "Dataset/PalmTest", "Dataset/PeaceTest", "Dataset/SwagTest", "Dataset/FistTest"]
TrainImages, TrainLabels = Add_Data(Train_Paths)
TestImages, TestLabels = Add_Data(Test_Paths)
TrainImages, TrainLabels = shuffle(TrainImages, TrainLabels, random_state=0)

In [11]:
#Train Data for Gesturized Mouse
model = CreateModel(len(Train_Paths))
history = model.fit(TrainImages, TrainLabels, n_epoch=50, validation_set = (TestImages, TestLabels), snapshot_step=100, show_metric=True, run_id='50412968')

Training Step: 3949  | total loss: 0.00328 | time: 253.747s
| Adam | epoch: 050 | loss: 0.00328 - acc: 0.9999 -- iter: 4992/5005
Training Step: 3950  | total loss: 0.00301 | time: 255.047s
| Adam | epoch: 050 | loss: 0.00301 - acc: 0.9999 | val_loss: 0.00110 - val_acc: 1.0000 -- iter: 5005/5005
--


In [12]:
#Save Model for Gesturized Mouse
model.save("Models\\swi_pa_pe_swa_f\\swi_pa_pe_swa_f.tfl")

INFO:tensorflow:C:\Users\saura\Downloads\Gesture Recognition\Models\swi_pa_pe_swa_f\swi_pa_pe_swa_f.tfl is not in all_model_checkpoint_paths. Manually adding it.


**Train Volume Bar Dataset**

In [18]:
# Add Data for Volume Bar
Train_Paths = ["Dataset/SwingImages", "Dataset/Swag"]
Test_Paths = ["Dataset/SwingTest","Dataset/SwagTest"]
TrainImages, TrainLabels = Add_Data(Train_Paths)
TestImages, TestLabels = Add_Data(Test_Paths)
TrainImages, TrainLabels = shuffle(TrainImages, TrainLabels, random_state=0)

In [19]:
#Train Data for Volume Bar
model = CreateModel(len(Train_Paths))
history2 = model.fit(TrainImages, TrainLabels, n_epoch=50, validation_set = (TestImages, TestLabels), snapshot_step=100, show_metric=True, run_id='50412968')

Training Step: 1599  | total loss: 0.27181 | time: 10.496s
| Adam | epoch: 050 | loss: 0.27181 - acc: 0.9882 -- iter: 1984/2002
Training Step: 1600  | total loss: 0.24463 | time: 11.868s
| Adam | epoch: 050 | loss: 0.24463 - acc: 0.9894 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 2002/2002
--


In [20]:
#Save Model for Gesturized Mouse
model.save("Models\\swing_swag\\swing_swag.tfl")

INFO:tensorflow:C:\Users\saura\Downloads\Gesture Recognition\Models\swing_swag\swing_swag.tfl is not in all_model_checkpoint_paths. Manually adding it.


**Test Volume Bar using Swing and Swag hand Gestures**


*   To start Testing press Key 'a' after recording windows appears
*   To stop Press Key 'q'



In [22]:
model = CreateModel(2)
model.load("Models/swing_swag/swing_swag.tfl")
Test_Model(model)

INFO:tensorflow:Restoring parameters from C:\Users\saura\Downloads\Gesture Recognition\Models\swing_swag\swing_swag.tfl


**Hand Gesturized Mouse Testing**


*   To start Testing press Key 'a' after recording windows appears
*   To stop Press Key 'q' 
*   If you see Percentage value greater than 99.5 percentage, Press Key 'd' (Key 'd' is given for extra protection for actions)



In [23]:
model = CreateModel(5)
model.load("Models\\swi_pa_pe_swa_f\\swi_pa_pe_swa_f.tfl")
Test_Model(model, actions = 1, class_names = ['Swing','Palm', 'Peace', 'Swag', 'Fist'])

INFO:tensorflow:Restoring parameters from C:\Users\saura\Downloads\Gesture Recognition\Models\swi_pa_pe_swa_f\swi_pa_pe_swa_f.tfl
